# Common Libraries

In [1]:
import os
os.environ["CUDA_HOME"] = "/usr/local/cuda-12.6" # for linux

import pickle
import numpy as np
import cupy as cp
import nibabel as nb
from glob import glob
from tqdm import trange
from numba import cuda
cuda.select_device(0)

from rsatoolbox.util.searchlight import get_volume_searchlight
#from nltools.data import Brain_Data
from collections import Counter
from itertools import combinations

In [2]:
from boostrsa.boostrsa_types import ShrinkageMethod
from boostrsa.cores.gpu.basic_operations import outer_sum_square, outer_sum

In [3]:
import sys
sys.path.append("/home/seojin/boostrsa/src/boostrsa/cores/cpgpu")

from stats import _covariance_diag

In [4]:
import sys
sys.path.append("/home/seojin/boostrsa/src/boostrsa/cores/cpu")
from mask import set_mask

In [5]:
import sys
sys.path.append("/home/seojin/Seojin_commonTool/Module")

from memory_management import convert_byte

# Params

In [6]:
subj_name = "DO01"
is_update_data = False

# Searchlight
radius = 3
threshold = 1

# Task params
n_split_data = 2048
n_thread_per_block = 1024
shrinkage_method = ShrinkageMethod.shrinkage_diag
n_block = int(np.ceil(n_split_data / n_thread_per_block))

# Constants

In [7]:
data_dir_path = f"/mnt/ext3/hanul/DO/stats/beta/{subj_name}/green_time"
full_mask_path = f"/mnt/ext3/hanul/DO/fMRI_data/proc_data/{subj_name}/full_mask.DO01.nii"
temp_dir_path = "/mnt/ext1/seojin/temp/test"

In [8]:
beta_paths = sorted(glob(os.path.join(data_dir_path, f"{subj_name}_beta_all_r*.nii")))
condition_paths = sorted(glob(os.path.join(data_dir_path, f"{subj_name}_conditions_r*.npy")))
residual_paths = sorted(glob(os.path.join(data_dir_path, f"{subj_name}_residual_r*.nii")))

In [9]:
if is_update_data:
    mask = nb.load(full_mask_path)
    betas = [Brain_Data(path, mask = mask) for path in beta_paths]
    conditions = [np.load(path, allow_pickle=True) for path in condition_paths]
    residuals = [Brain_Data(path, mask = mask) for path in residual_paths]

    for i, beta in enumerate(betas):
        np.save(os.path.join(temp_dir_path, f"beta_r{i+1}.npy"), beta.data)
    
    for i, residual in enumerate(residuals):
        np.save(os.path.join(temp_dir_path, f"residual_r{i+1}.npy"), residual.data)
    
    for i, condition in enumerate(conditions):
        np.save(os.path.join(temp_dir_path, f"condition_r{i+1}.npy"), condition)
else:
    betas = [np.load(f"/mnt/ext1/seojin/temp/test/beta_r{i+1}.npy", allow_pickle = True) for i in range(0, 1)]
    conditions = [np.load(f"/mnt/ext1/seojin/temp/test/condition_r{i+1}.npy", allow_pickle = True) for i in range(0, 1)]
    residuals = [np.load(f"/mnt/ext1/seojin/temp/test/residual_r{i+1}.npy", allow_pickle = True) for i in range(0, 1)]

# Test - GPU masking

In [10]:
"""i = 0
n_channel = len(mask_1d_indexes)
n_block = int(np.ceil(n_split_data / n_thread_per_block))

target_neighbors_ = target_neighbors[i:i + n_split_data, :]
n_neighbor = target_neighbors.shape[1]

n_target_center = len(target_neighbors_)
mask_out = cuda.to_device(np.zeros((n_target_center, n_channel)))
set_mask[n_block, n_thread_per_block](target_neighbors_, mask_1d_indexes, mask_out)
memory_usage = memory(mask_out.nbytes, "GB")
print(f"{memory_usage}GB")"""

'i = 0\nn_channel = len(mask_1d_indexes)\nn_block = int(np.ceil(n_split_data / n_thread_per_block))\n\ntarget_neighbors_ = target_neighbors[i:i + n_split_data, :]\nn_neighbor = target_neighbors.shape[1]\n\nn_target_center = len(target_neighbors_)\nmask_out = cuda.to_device(np.zeros((n_target_center, n_channel)))\nset_mask[n_block, n_thread_per_block](target_neighbors_, mask_1d_indexes, mask_out)\nmemory_usage = memory(mask_out.nbytes, "GB")\nprint(f"{memory_usage}GB")'

# Data configuration

In [11]:
is_update_neighbors = False

In [12]:
# Mask
mask = nb.load(full_mask_path)
nx, ny, nz = mask.get_fdata().shape
mask_3d_indexes = np.array(np.where(mask.get_fdata() == 1))
mask_1d_indexes = np.ravel_multi_index(mask_3d_indexes, (nx, ny, nz))

# Neighbors
if is_update_neighbors:
    centers, neighbors = get_volume_searchlight(mask.get_fdata(), 
                                                radius=radius, 
                                                threshold=threshold)
    np.save(os.path.join(temp_dir_path, "centers.npy"), centers)
    with open(os.path.join(temp_dir_path, "neighbors"),"wb") as f:
        pickle.dump(neighbors, f)
else:
    centers = np.load(os.path.join(temp_dir_path, "centers.npy"))
    with open(os.path.join(temp_dir_path, "neighbors"),"rb") as f:
        neighbors = pickle.load(f)
    
n_neighbors = np.array([len(e) for e in neighbors])
uq_n_neighbors = np.unique([len(e) for e in neighbors])

flags = (n_neighbors == Counter(n_neighbors).most_common()[0][0])
centers = centers[flags]
neighbors = [e for e, f in zip(neighbors, flags) if f]
neighbors = np.array(neighbors)
n_neighbor = neighbors.shape[-1]

In [13]:
sim_run = 12

# Betas
beta = []
for i in range(1, sim_run):
    beta.append(betas[0])
beta = np.concatenate(beta)

# Condition
condition = []
for i in range(1, sim_run):
    condition.append(conditions[0])
condition = np.concatenate(condition)

# Residuals
residual = np.expand_dims(residuals[0], axis = 0)
for i in range(1, sim_run):
    residual = np.r_[residual, np.expand_dims(residuals[0], axis = 0)]

n_run, n_point, n_channel = residual.shape
n_cond = beta.shape[0]

# Sessions
sessions = np.concatenate([np.repeat(run_number, condition.shape[0] / n_run) for run_number in range(1, n_run + 1)])
uq_sessions = np.unique(sessions)

In [14]:
dtype = np.float32

In [15]:
residual = residual.astype(dtype)
beta = beta.astype(dtype)

In [16]:
fold_info = list(combinations(np.arange(len(uq_sessions)), 2))
n_fold = len(fold_info)

# Split mask

In [17]:
target_centers = centers[:n_split_data]
target_neighbors = neighbors[:n_split_data]
split_mask = set_mask(target_neighbors, mask_1d_indexes)

# Masked

In [18]:
masked_measurements = np.empty((n_split_data, n_cond, n_neighbor), dtype = beta.dtype)
for j, m in enumerate(split_mask):
    masked_measurements[j] = beta[:, m == 1]
masked_measurements = cp.asarray(masked_measurements)

In [19]:
masked_residuals = np.empty((n_split_data, n_run, n_point, n_neighbor),
                            dtype=residual.dtype)
for i, m in enumerate(split_mask):
    masked_residuals[i] = residual[:, :, m == 1]

# Preprocessing

In [54]:
# Calculate demean
target_residuals = masked_residuals.reshape(-1, n_point, n_neighbor)
mean_residuals = np.mean(target_residuals, axis = 1, keepdims=1)
target_residuals = (target_residuals - mean_residuals)

# Covariance

In [55]:
n_processing_unit = target_residuals.shape[0]

In [57]:
out_sum_device = cuda.to_device(np.zeros((n_processing_unit, n_neighbor, n_neighbor), dtype = dtype))
print(convert_byte(out_sum_device.nbytes, "GB"), "GB")

outer_sum[n_block, n_thread_per_block](target_residuals, out_sum_device)
outer_sum_result = out_sum_device.copy_to_host()
del out_sum_device

0.3959197998046875 GB


/home/seojin/anaconda3/envs/boostrsa/lib/python3.9/site-packages/numba/cuda/dispatcher.py:538: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/seojin/anaconda3/envs/boostrsa/lib/python3.9/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [87]:
out_sum_square_device = cuda.to_device(np.zeros((n_processing_unit, n_neighbor, n_neighbor), dtype = dtype))
print(convert_byte(out_sum_square_device.nbytes, "GB"), "GB")

outer_sum_square[n_block, n_thread_per_block](target_residuals, out_sum_square_device)
outer_sum_square_result = out_sum_square_device.copy_to_host()
del out_sum_square_device

0.791839599609375 GB


/home/seojin/anaconda3/envs/boostrsa/lib/python3.9/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [27]:
s = outer_sum_result / n_point
s2 = outer_sum_square_result / n_point
b2 = np.sum(s2 - s * s, axis = (1, 2)) / n_point

In [28]:
repeat_eyes = np.repeat(np.eye(n_neighbor)[:, :, np.newaxis], n_processing_unit, axis = 2).T

In [29]:
diag_s = np.diagonal(s, axis1 = 1, axis2 = 2)
m = (np.sum(diag_s, axis = 1) / n_neighbor)
d2 = np.sum((s - m[:, None, None] * repeat_eyes) ** 2, axis = (1, 2))

In [30]:
b2 = np.minimum(d2, b2)

In [31]:
s_shrink = (b2 / d2 * m)[:, None, None] * repeat_eyes + ((d2-b2) / d2)[:, None, None] * s

/tmp/ipykernel_2353319/1731073590.py:1: RuntimeWarning: invalid value encountered in divide
  s_shrink = (b2 / d2 * m)[:, None, None] * repeat_eyes + ((d2-b2) / d2)[:, None, None] * s


In [32]:
dof = n_point - 1
s_shrink = s_shrink * n_point / dof

In [34]:
precisions = cp.linalg.inv(cp.asarray(s_shrink)).get()

In [35]:
r, c = np.triu_indices(n_neighbor, k = 0)

In [39]:
precisions = precisions[ :, r, c]

In [40]:
mempool = cp.get_default_memory_pool()
mempool.free_all_blocks()

In [90]:
np.save(os.path.join(temp_dir_path, "precision.npy"), precisions)

# Fold precision

In [131]:
dtype = np.float32

In [162]:
precisions = np.load(os.path.join(temp_dir_path, "precision.npy"))
precisions = precisions.astype(dtype)

In [134]:
from boostrsa.cores.cpu.matrix import convert_1d_to_symmertic, mean_fold_variance

In [149]:
type(cp.asarray(precisions))

cupy.ndarray

In [161]:
precisions.shape

(24576, 4371)

In [166]:
import cupyx.scipy.sparse as cpxs

# Convert upper triangular storage into a sparse format
sparse_upper = cpxs.csr_matrix(cp.asarray(precisions))

In [171]:
inv_sparse = cpxs.linalg.inv(sparse_upper)

AttributeError: module 'cupyx.scipy.sparse' has no attribute 'linalg'

In [167]:
sparse_upper

In [164]:
# Compute Cholesky factorization (for positive definite matrices)
L = cp.linalg.cholesky(cp.asarray(precisions))

# Invert using Cholesky factors
L_inv = cp.linalg.inv(L)

# Compute inverse using (L^T L)^(-1)
inv_matrix = L_inv.T @ L_inv

print("Inverse Matrix:\n", inv_matrix)

LinAlgError: Last 2 dimensions of the array must be square

In [160]:
precisions.shape

(24576, 4371)

In [159]:
x = cp.linalg.solve(cp.asarray(precisions), 0)

LinAlgError: cupy.linalg only supports cupy.ndarray

In [140]:
import cupy as cp

# Assume upper_triangular_matrix is stored in a compressed form
upper_triangular_matrix = cp.array(target_precs)  # Only upper triangle stored

# Perform Cholesky decomposition (works only for positive definite matrices)
L = cp.linalg.cholesky(upper_triangular_matrix)

# Compute inverse using the triangular matrix
L_inv = cp.linalg.inv(L)
inv_matrix = L_inv.T @ L_inv  # (L^T L)^(-1) = A^(-1)

LinAlgError: Last 2 dimensions of the array must be square

In [138]:
# Upper triangle value만 가지고 있는 precision matrix 대상으로 invert matrix를 계산할 수 있는 코드 구현

In [135]:
prec_mat_shape = int((n_neighbor * n_neighbor - n_neighbor) / 2) + n_neighbor
target_precs = precisions.reshape(-1, prec_mat_shape)
target_precs = np.array([convert_1d_to_symmertic(pre, size = n_neighbor).astype(dtype) for pre in target_precs])
variances = cp.linalg.inv(cp.asarray(target_precs))

In [23]:
variances = variances.reshape(len(target_centers), n_run, n_neighbor, n_neighbor).get()

In [24]:
mempool = cp.get_default_memory_pool()
mempool.free_all_blocks()

In [127]:
fold_preicions = cp.linalg.inv(cp.asarray(mean_fold_variance(variances, fold_info)))
fold_preicions = fold_preicions.get()

In [128]:
mempool.free_all_blocks()

In [129]:
fold_preicions = fold_preicions.reshape(len(target_centers), len(fold_info), n_neighbor, n_neighbor)

In [130]:
r, c = np.triu_indices(n_neighbor, k = 0)
fold_preicions = fold_preicions[:, :, r, c].copy()

KeyboardInterrupt: 

In [125]:
from time import time

start_time = time()
fold_preicions = np.ascontiguousarray(fold_preicions)
end_time = time()

print(end_time - start_time)

101.57743692398071


In [ ]:
fold_preicions = np.asfortranarray(fold_preicions)

In [102]:
fold_preicions.shape

(2048, 66, 4371)

In [103]:
fold_preicions = cuda.to_device(fold_preicions)

In [104]:
convert_byte(fold_preicions.nbytes, "GB")

2.2009735107421875

In [107]:
fold_preicions.shape

(2048, 66, 4371)

In [108]:
B = np.array([[2, 3, 4],
              [3, 5, 6],
              [4, 6, 8]])

In [112]:
r, c = np.triu_indices(3, k = 0)


In [113]:
r

array([0, 0, 0, 1, 1, 2])

In [114]:
B[r, c]

array([2, 3, 4, 5, 6, 8])

In [115]:
# 내가 어떤 부분에 접근해야하는지를 알면 연산 가능!

# AVG measurement per session

In [31]:
# Avg conds per session
avg_measurements = []
avg_conds = []
for session in uq_sessions:
    filtering_session = sessions == session
    sess_cond = condition[filtering_session]
    sess_measurements = cp.compress(filtering_session, masked_measurements, axis = 1)
    
    mean_measurments = []
    for cond in np.unique(condition):
        filtering_cond = (sess_cond == cond)
        cond_measurments = cp.compress(filtering_cond, sess_measurements, axis = 1)
        mean_cond_measurement = cp.mean(cond_measurments, axis = 1)
        mean_measurments.append(cp.expand_dims(mean_cond_measurement, axis = 1))

        avg_conds.append(cond)

    avg_measurements.append(cp.expand_dims(cp.concatenate(mean_measurments, axis = 1), axis = 1))
avg_measurements = cp.concatenate(avg_measurements, axis = 1).get()

avg_conds = np.array(avg_conds)

mempool.free_all_blocks()

In [42]:
convert_byte(2048 * 66 * 396 * 93 * 32, "GB")

148.35498046875

In [34]:
a = np.zeros((len(target_centers), n_fold, n_cond, n_neighbor), dtype = dtype)

In [51]:
a.nbytes

19911868416

In [52]:
convert_byte(a.nbytes, "GB")

18.54437255859375

In [50]:
# make kernel
avg_measurements = cuda.to_device(avg_measurements)

matmul1_out = cuda.to_device(np.zeros((len(target_centers), n_fold, n_cond, n_neighbor), dtype = dtype))
kernel_out = cuda.to_device(np.zeros((len(target_centers), n_fold, n_cond, n_cond), dtype = dtype))
calc_kernel[block_2ds, thread_2ds](avg_measurements, fold_preicions, fold_info, matmul1_out, kernel_out)

CudaAPIError: [2] Call to cuMemAlloc results in CUDA_ERROR_OUT_OF_MEMORY

In [51]:
del matmul1_out

In [45]:
len(target_centers), n_fold, n_cond, n_neighbor

(2048, 66, 396, 93)

18.54437255859375